**IMPORTING NECESSARY PACKAGAES**

In [91]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.api._v2.keras.layers import Dense, Activation, Dropout
from keras.api._v2.keras.optimizers import Adam
from keras.api._v2.keras.metrics import Accuracy
from keras.api._v2.keras.models import Sequential
from keras.api._v2.keras.layers import Dense, Dropout

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score,roc_curve,confusion_matrix

from csv import writer
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

**LOADING THE DATA**

In [92]:
insta_df_train=pd.read_csv('./train.csv')
insta_df_train

,nums/length username,nums/length fullname,name==username,description length,#posts,#followers,#follows,fake
0,0.27,0.00,0,53,32,1000,955,0
1,0.00,0.00,0,44,286,2740,533,0
2,0.10,0.00,0,0,13,159,98,0
3,0.00,0.00,0,82,679,414,651,0
4,0.00,0.00,0,0,6,151,126,0
...,...,...,...,...,...,...,...,...
571,0.55,0.44,0,0,33,166,596,1
572,0.38,0.33,0,21,44,66,75,1
573,0.57,0.00,0,0,4,96,339,1
574,0.57,0.00,0,11,0,57,73,1


In [93]:
insta_df_test=pd.read_csv("./test.csv")
insta_df_test

,nums/length username,nums/length fullname,name==username,description length,#posts,#followers,#follows,fake
0,0.33,0.33,1.0,30.0,35.0,488.0,604.0,0.0
1,0.00,0.00,0.0,64.0,3.0,35.0,6.0,0.0
2,0.00,0.00,0.0,82.0,319.0,328.0,668.0,0.0
3,0.00,0.00,0.0,143.0,273.0,14890.0,7369.0,0.0
4,0.50,0.00,0.0,76.0,6.0,225.0,356.0,0.0
...,...,...,...,...,...,...,...,...
139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
insta_df_test.tail(1)

,nums/length username,nums/length fullname,name==username,description length,#posts,#followers,#follows,fake
143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**DATA ANALYSIS**

In [95]:
insta_df_train.head()

,nums/length username,nums/length fullname,name==username,description length,#posts,#followers,#follows,fake
0,0.27,0.0,0,53,32,1000,955,0
1,0.00,0.0,0,44,286,2740,533,0
2,0.10,0.0,0,0,13,159,98,0
3,0.00,0.0,0,82,679,414,651,0
4,0.00,0.0,0,0,6,151,126,0


In [96]:
insta_df_train.tail()

,nums/length username,nums/length fullname,name==username,description length,#posts,#followers,#follows,fake
571,0.55,0.44,0,0,33,166,596,1
572,0.38,0.33,0,21,44,66,75,1
573,0.57,0.00,0,0,4,96,339,1
574,0.57,0.00,0,11,0,57,73,1
575,0.27,0.00,0,0,2,150,487,1


In [97]:
insta_df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   nums/length username  576 non-null    float64
 1   nums/length fullname  576 non-null    float64
 2   name==username        576 non-null    int64  
 3   description length    576 non-null    int64  
 4   #posts                576 non-null    int64  
 5   #followers            576 non-null    int64  
 6   #follows              576 non-null    int64  
 7   fake                  576 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 36.1 KB


In [98]:
insta_df_train.describe()

,nums/length username,nums/length fullname,name==username,description length,#posts,#followers,#follows,fake
count,576.000000,576.000000,576.000000,576.000000,576.000000,5.760000e+02,576.000000,576.000000
mean,0.163837,0.036094,0.034722,22.623264,107.489583,8.530724e+04,508.381944,0.500000
std,0.214096,0.125121,0.183234,37.702987,402.034431,9.101485e+05,917.981239,0.500435
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,3.900000e+01,57.500000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,9.000000,1.505000e+02,229.500000,0.500000
75%,0.310000,0.000000,0.000000,34.000000,81.500000,7.160000e+02,589.500000,1.000000
max,0.920000,1.000000,1.000000,150.000000,7389.000000,1.533854e+07,7500.000000,1.000000


In [99]:
insta_df_train.isnull().sum()

nums/length username    0
nums/length fullname    0
name==username          0
description length      0
#posts                  0
#followers              0
#follows                0
fake                    0
dtype: int64

In [100]:
insta_df_train['fake'].value_counts()

fake
0    288
1    288
Name: count, dtype: int64

**DATA MODELLING**

In [101]:
X_train = insta_df_train.drop(columns = ['fake'])
X_test = insta_df_test.drop(columns = ['fake'])
X_train

,nums/length username,nums/length fullname,name==username,description length,#posts,#followers,#follows
0,0.27,0.00,0,53,32,1000,955
1,0.00,0.00,0,44,286,2740,533
2,0.10,0.00,0,0,13,159,98
3,0.00,0.00,0,82,679,414,651
4,0.00,0.00,0,0,6,151,126
...,...,...,...,...,...,...,...
571,0.55,0.44,0,0,33,166,596
572,0.38,0.33,0,21,44,66,75
573,0.57,0.00,0,0,4,96,339
574,0.57,0.00,0,11,0,57,73


In [102]:
y_train = insta_df_train['fake']
y_test = insta_df_test['fake']
y_train

0      0
1      0
2      0
3      0
4      0
      ..
571    1
572    1
573    1
574    1
575    1
Name: fake, Length: 576, dtype: int64

In [103]:
from sklearn.preprocessing import StandardScaler

scaler_x = StandardScaler()
X_train = scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)

In [114]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes = 2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 2)

c:\Users\thegr\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\series.py:953: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


IndexError: index -2147483648 is out of bounds for axis 1 with size 2

In [ ]:
y_train

0      0
1      0
2      0
3      0
4      0
      ..
571    1
572    1
573    1
574    1
575    1
Name: fake, Length: 576, dtype: int64

In [ ]:
y_test

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
139    NaN
140    NaN
141    NaN
142    NaN
143    NaN
Name: fake, Length: 144, dtype: float64

In [ ]:

model = Sequential()
model.add(Dense(50, input_dim=7, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2,activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 50)                400       
                                                                 
 dense_16 (Dense)            (None, 150)               7650      
                                                                 
 dropout_9 (Dropout)         (None, 150)               0         
                                                                 
 dense_17 (Dense)            (None, 150)               22650     
                                                                 
 dropout_10 (Dropout)        (None, 150)               0         
                                                                 
 dense_18 (Dense)            (None, 25)                3775      
                                                                 
 dropout_11 (Dropout)        (None, 25)               

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [115]:
epochs_hist = model.fit(X_train, y_train, epochs = 50,  verbose = 1, validation_split = 0.1)

Epoch 1/50


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [ ]:
print(epochs_hist.history.keys())


In [ ]:
X_test

array([[ 0.77679158,  2.35102654,  5.27257053, ..., -0.18046362,
        -0.09327375,  0.10425177],
       [-0.76591601, -0.28872256, -0.18966081, ..., -0.26012798,
        -0.0937719 , -0.54774388],
       [-0.76591601, -0.28872256, -0.18966081, ...,  0.52655754,
        -0.0934497 ,  0.17403056],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]])

In [ ]:
predicted = model.predict(X_test)

5/5 [==============================] - 0s 3ms/step


In [ ]:
ans=predicted[-1]
ans

array([nan, nan], dtype=float32)

In [ ]:
array=np.array(ans)
integer_array=np.round(array).astype(int)
integer_array=integer_array[1:]
print(integer_array)


[-2147483648]


C:\Users\thegr\AppData\Local\Temp\ipykernel_9980\3689161420.py:2: RuntimeWarning: invalid value encountered in cast
  integer_array=np.round(array).astype(int)


In [ ]:
if(integer_array==0):
  print("It is a real account")
else:
  print("It is a fake account")

It is a fake account
